# 10X Dose Analysis: All Task Conditions

Comprehensive analysis of behavioral features for 10X dose experiments across all task conditions.
Analyzes velocity, curvature, and angle features for each task: AllTask, FoodOnly, FoodLight, ToyOnly, ToyLight, LightOnly.

## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]
print(project_root)
sys.path.append(str(project_root))

## Load data

In [ ]:
import importlib
import Python_scripts.config as config_mod
importlib.reload(config_mod)
from Python_scripts.config import load_dlc_table
import pandas as pd

# Load dlc_table.csv
dlc_table = load_dlc_table()
print(f"[✓] Loaded dlc_table with {len(dlc_table)} trials")
print(f"[INFO] Columns: {list(dlc_table.columns)}")

## Import modules

In [ ]:
import importlib
import Python_scripts.Data_analysis.fetch_id_list as fetch_mod
importlib.reload(fetch_mod)
from Python_scripts.Data_analysis.fetch_id_list import fetch_id_list

In [ ]:
import importlib
import Python_scripts.Feature_functions.motion_features
import Python_scripts.Feature_functions.trajectory_curvature
import Python_scripts.Feature_functions.angle_features
import Python_scripts.Feature_functions.db_utils
import Python_scripts.Data_analysis.plot_groupwise_bar
import Python_scripts.Data_analysis.normalized_bodypart

importlib.reload(Python_scripts.Feature_functions.db_utils)
importlib.reload(Python_scripts.Data_analysis.normalized_bodypart)
importlib.reload(Python_scripts.Feature_functions.motion_features)
importlib.reload(Python_scripts.Feature_functions.trajectory_curvature)
importlib.reload(Python_scripts.Feature_functions.angle_features)
importlib.reload(Python_scripts.Data_analysis.plot_groupwise_bar)

from Python_scripts.Feature_functions.motion_features import (
    compute_motion_features_per_minute, batch_compute_motion_features_per_minute
)
from Python_scripts.Feature_functions.trajectory_curvature import (
    compute_trajectory_curvature, batch_trajectory_curvature
)
from Python_scripts.Feature_functions.angle_features import (
    angle_features_for_trial, batch_angle_features

)
from Python_scripts.Data_analysis.plot_groupwise_bar import plot_groupwise_bar

## Analysis loop over all tasks

Run all three analyses (velocity, curvature, angle) for each task condition.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define all task conditions to analyze
task_conditions = [None, 'FoodOnly', 'FoodLight', 'ToyOnly', 'ToyLight', 'LightOnly']

# Fixed parameters for analyses
curvature_params = {'bodypart': 'Midback', 'window': 23}
velocity_params = {'bodypart': 'Head', 'time_limit': None, 'smooth': False, 'window': 5}
angle_params = {'smooth_window': None, 'likelihood_threshold': 0.65}

# Store all results
all_results = []

for task_name in task_conditions:
    task_label = "AllTask" if task_name is None else task_name
    print(f"\n{'='*60}")
    print(f"Processing task: {task_label}")
    print(f"{'='*60}")
    
    # Fetch trial IDs for this task
    saline_id, ghrelin_id, exc_id, inh_id = fetch_id_list(
        dlc_table,
        task_name=task_name,
        dose_mult=10,
        genotype="white",
        min_trial_length=None
    )
    
    print(f"saline_id: {len(saline_id)} trials")
    print(f"ghrelin_id: {len(ghrelin_id)} trials")
    
    group_specs = {"Saline": saline_id, "Ghrelin": ghrelin_id}
    group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}
    
    if not group_specs:
        print(f"[WARN] No groups with data for {task_label}, skipping...")
        continue
    
    # ===== CURVATURE ANALYSIS =====
    print(f"\n[INFO] Running curvature analysis for {task_label}...")
    frames = []
    for label, ids in group_specs.items():
        df = batch_trajectory_curvature(
            dlc_table, ids, **curvature_params
        )
        df["group"] = label
        df['task'] = task_label
        frames.append(df)
    
    if frames:
        df_curv = pd.concat(frames, ignore_index=True)
        order = list(group_specs.keys())
        fig, ax, stats_df = plot_groupwise_bar(
            df_curv, y='mean_curvature',
            ylabel='Mean curvature',
            plot_type='bar', show_points=True,
            order=order
        )
        ax.set_title(f"{task_label} | Curvature | window={curvature_params['window']}", pad=20)
        fig.savefig(f"White_10X_{task_label}_curvature.pdf", dpi=300, bbox_inches='tight')
        plt.close(fig)
        print(f"[✓] Saved White_10X_{task_label}_curvature.pdf")
        print(stats_df)
        all_results.append(('curvature', task_label, df_curv))
    
    # ===== VELOCITY ANALYSIS =====
    print(f"\n[INFO] Running velocity analysis for {task_label}...")
    frames = []
    for label, ids in group_specs.items():
        df = batch_compute_motion_features_per_minute(
            dlc_table, ids, **velocity_params
        )
        df = df[['trial_id', 'velocity_per_min']].copy().dropna()
        df['group'] = label
        df['task'] = task_label
        frames.append(df)
    
    if frames:
        df_vel = pd.concat(frames, ignore_index=True)
        order = list(group_specs.keys())
        fig, ax, stats_df = plot_groupwise_bar(
            df_vel, y='velocity_per_min',
            ylabel='Average speed (units/min)',
            plot_type='bar', show_points=True,
            order=order
        )
        ax.set_title(f"{task_label} | Velocity | window={velocity_params['window']}", pad=20)
        fig.savefig(f"White_10X_{task_label}_velocity.pdf", dpi=300, bbox_inches='tight')
        plt.close(fig)
        print(f"[✓] Saved White_10X_{task_label}_velocity.pdf")
        print(stats_df)
        all_results.append(('velocity', task_label, df_vel))

    # ===== ANGLE FEATURES ANALYSIS =====
    print(f"\n[INFO] Running angle analysis for {task_label}...")
    frames = []
    for label, ids in group_specs.items():
        df = batch_angle_features(
            dlc_table, ids, **angle_params
        )
        df = df[['trial_id', 'head_body_misalignment_p95']].copy().dropna()
        df['group'] = label
        df['task'] = task_label
        frames.append(df)
    
    if frames:
        df_ang = pd.concat(frames, ignore_index=True)
        order = list(group_specs.keys())
        fig, ax, stats_df = plot_groupwise_bar(
            df_ang, y='head_body_misalignment_p95',
            ylabel='Mean head_body_misalignment_p95',
            plot_type='bar', show_points=True,
            order=order
        )
        ax.set_title(f"{task_label} | Angle | likelihood={angle_params['likelihood_threshold']}", pad=20)
        fig.savefig(f"White_10X_{task_label}_angle.pdf", dpi=300, bbox_inches='tight')
        plt.close(fig)
        print(f"[✓] Saved White_10X_{task_label}_angle.pdf")
        print(stats_df)
        all_results.append(('angle', task_label, df_ang))

print(f"\n{'='*60}")
print(f"Analysis complete! Processed {len(task_conditions)} task conditions.")
print(f"{'='*60}")

## Optional: Export combined results to Excel

In [ ]:
# # Export combined results to Excel
# for feature_type, task_label, df_result in all_results:
#     # Get trial IDs and determine ID column name
#     if feature_type == 'curvature':
#         trial_ids = df_result['id'].unique().tolist()
#         id_col = 'id'
#     else:
#         trial_ids = df_result['trial_id'].unique().tolist()
#         id_col = 'trial_id'
    
#     # Fetch metadata from dlc_table DataFrame
#     meta_df = dlc_table[dlc_table['id'].isin(trial_ids)][['id', 'task', 'modulation']].copy()
    
#     # Merge and save
#     df_out = df_result.merge(meta_df, left_on=id_col, right_on='id', how='left')
#     if 'id' in df_out.columns and id_col != 'id':
#         df_out = df_out.drop(columns=['id'])
    
#     filename = f"10X_White_{task_label}_{feature_type}.xlsx"
#     df_out.to_excel(filename, index=False)
#     print(f"[✓] Saved {filename}")